In [4]:
import unicodedata
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
import re

def remove_diacritics(text):
    # Chuẩn hóa Unicode và loại bỏ dấu
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    # Loại bỏ ký tự không phải chữ cái/số nếu cần
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower()

# Khởi tạo SparkSession
spark = SparkSession.builder.master("local").appName("Skills Standard").getOrCreate()

# Đọc dữ liệu từ file CSV
file1 = spark.read.option("header", "true").csv("./skills.csv")
file2 = spark.read.option("header", "true").csv("./combined_output_no_url.csv")

# Chuẩn bị file1 với cột skill và standard
file1 = file1.select("skill", "standard")

# Chuyển đổi skill thành dạng bảng với mỗi skill và standard
file1 = file1.withColumn("skill", F.lower(col("skill")))

# Broadcast file1 (để tránh lỗi về SparkContext)
broadcast_file1 = spark.sparkContext.broadcast(file1.collect())

# Hàm tạo ra trường skills_standard
# Hàm tạo ra trường skills_standard với kiểm tra None hoặc rỗng
def add_skills_standard(job_skills):
    if job_skills is None or job_skills.strip() == "":  # Kiểm tra nếu job_skills là None hoặc rỗng
        return "None"
    
    skills_list = job_skills.lower()
    standard_list = []
    
    # Duyệt qua từng kỹ năng và tìm kiếm trong broadcast_file1
    # for skill in skills_list:
    #     skill = skill.strip().lower()
        
        # Tìm kiếm standard cho skill trong broadcast data
    for row in broadcast_file1.value:
        if row['skill'].lower() in skills_list:
            standard_list.append(row['standard'])
    my_set = set(standard_list)
    return ",".join(my_set)

# Đăng ký hàm UDF (User Defined Function)
add_skills_standard_udf = F.udf(add_skills_standard, StringType())

# Thêm cột skills_standard vào file2
file2 = file2.withColumn("skills_standard", add_skills_standard_udf(col("job_skills")))

# Đếm số dòng có trường skills_standard rỗng hoặc null
count_null_or_empty = file2.filter((col("skills_standard").isNull()) | (col("skills_standard") == "")).count()

# Đếm tổng số dòng
total_rows = file2.count()

# Tính tỷ lệ phần trăm số dòng có skills_standard rỗng
percentage_empty = (count_null_or_empty / total_rows) * 100

# In kết quả
print(f"Số dòng có skills_standard rỗng: {count_null_or_empty}")
print(f"Tổng số dòng: {total_rows}")
print(f"Tỷ lệ phần trăm dòng có skills_standard rỗng: {percentage_empty:.2f}%")
# Số dòng có skills_standard rỗng: 3431
# Tổng số dòng: 114462
# Tỷ lệ phần trăm dòng có skills_standard rỗng: 3.00%


Số dòng có skills_standard rỗng: 280
Tổng số dòng: 114462
Tỷ lệ phần trăm dòng có skills_standard rỗng: 0.24%


In [ ]:
# # Lọc các dòng có skills_standard rỗng hoặc null
df = file2.na.replace("", "None", subset=["skills_standard"])
# empty_skills_standard_df = file2.filter((col("skills_standard").isNull()) | (col("skills_standard") == ""))

# # Ghi ra file CSV mới
df.write.option("header", "true").csv("empty_skills_standard_output")


In [13]:

df.select("skills_standard").take(20)

[Row(skills_standard='Management Skills,Training,Office,Planning,Budget Estimation,Project Management'),
 Row(skills_standard='Good health,Responsibility,Hardworking,Honesty,Attention to Detail,Business Skills'),
 Row(skills_standard='Veterinary,Persuasion,Sale,Honesty,Good Health,University Graduate,Business Skills,Communication Skills,Sales Skills'),
 Row(skills_standard='Independent Work,Responsibility,College Graduate,Progressive,Hardworking,Honesty,University Graduate,Civil Engineering,XD,Passion,HR'),
 Row(skills_standard='Persuasion,Enthusiastic Learner,Good-looking,Hardworking,Has Smartphone,Real Estate,Business Skills,Communication Skills,Sales Skills,None,Negotiation'),
 Row(skills_standard='College Graduate,Maintenace,Electromechanics,University Graduate,Construction'),
 Row(skills_standard='Misa,Secondary School Graduate'),
 Row(skills_standard='Teamwork,Good health,Responsibility,Management Skills,Supervision,Planning,Environmental,Problem Solving,Honesty,Attention to Deta

In [15]:
none_skills_standard_df = df.filter((col("skills_standard") == "None"))
print(none_skills_standard_df.count())

17562


In [16]:
blank_skills_standard_df = df.filter((col("skills_standard") == ""))
print(blank_skills_standard_df.count())

0
